### Load the Commit Statistics
For this clustering we examined 39 GitHub projects containing a total of 2,629 commits. The projects all included the Fluence Marine library. Code for Fluence is generally written in React and Rust.

In [1]:
project_root_path = '../../..'

import sys
bob_lib_path = project_root_path + "/sandbox/python/bob/"
sys.path.append(bob_lib_path)

import commit_stat_engineering
import json

dataset_dir = project_root_path + '/data/github/2022-04-07-marine-search'
project_stats_dir = dataset_dir + '/projects'
all_commit_logs = commit_stat_engineering.find_files('commit-stat.log.json', project_stats_dir)
all_stats = commit_stat_engineering.get_stats_for_all_commits(all_commit_logs)

### Commit Stats


In [2]:
print(all_stats[0].keys())

dict_keys(['commit', 'totalFiles', 'binFiles', 'textFiles', 'textLines', 'javascriptFiles', 'javascriptLines', 'javascriptFilePct', 'javascriptLinePct', 'rustFiles', 'rustLines', 'rustFilePct', 'rustLinePct', 'markdownFiles', 'markdownLines', 'markdownFilePct', 'markdownLinePct', 'jsonFiles', 'jsonLines', 'jsonFilePct', 'jsonLinePct', 'imgFiles', 'imgFilePct', 'lockFiles', 'lockLines', 'lockFilePct', 'lockLinePct', 'yarnFiles', 'yarnLines', 'yarnFilePct', 'yarnLinePct', 'htmlFiles', 'htmlLines', 'htmlFilePct', 'htmlLinePct', 'clojureFiles', 'clojureLines', 'clojureFilePct', 'clojureLinePct', 'shellFiles', 'shellLines', 'shellFilePct', 'shellLinePct', 'gitignoreFiles', 'gitignoreLines', 'gitignoreFilePct', 'gitignoreLinePct', 'noextbinFiles', 'noextbinFilePct', 'noexttextFiles', 'noexttextLines', 'noexttextFilePct', 'noexttextLinePct'])


### Load Labels

In [3]:
import pandas as pd

data = pd.read_csv(project_root_path + '/sandbox/data/bob/commit_labeling.csv', sep='\t')
data

,Unnamed: 0,wtf,is_bulk,is_minor,is_artifact,js_minor,js_std,rust_minor,rust_std,rust_pack,...,textFiles,textLines,javascriptLinePct,rustLinePct,markdownLinePct,jsonLinePct,imgFiles,lockLinePct,yarnLinePct,htmlLinePct
0,361427f483963aebd2cd3f9114909383f7c69056,1,0,0,0,0,0,1,0,0,...,1,6,0.00,100.0,0.00,0.00,0,0.0,0.00,0.0
1,b3a8f3cc31ba8a87270e85d9f5026d784f01942e,1,0,0,0,0,0,1,0,0,...,2,6,0.00,100.0,0.00,0.00,0,0.0,0.00,0.0
2,e5f96c9948e9aa0cc5f35ca766d07056d8e9fd48,1,0,0,0,0,0,1,0,0,...,1,5,0.00,100.0,0.00,0.00,0,0.0,0.00,0.0
3,49dcc227721fd2d19610797c2f43149658ee76e5,1,0,0,0,0,0,1,0,0,...,1,2,0.00,100.0,0.00,0.00,0,0.0,0.00,0.0
4,0e281231a1c92cf065b46f5eaf6e48801a8bca6e,1,0,0,0,1,0,0,0,0,...,2,6,83.33,0.0,16.66,0.00,0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,ed972d32d8e5b68a3e230080cfc6342ef003420c,1,1,0,0,0,0,0,0,0,...,14,5495,2.76,0.0,0.00,88.98,0,0.0,0.00,0.0
136,6c5397539882294dffb62eacd265a303b0c33873,1,1,0,0,0,0,0,0,0,...,10,4109,21.19,0.0,1.63,77.17,0,0.0,0.00,0.0
137,24dbadaa7ca32b891e3c28cc8024153d2304e84f,1,1,0,0,0,0,0,0,0,...,10,501,20.75,0.0,14.77,54.29,0,0.0,0.39,0.0
138,90a25bedfeb738a9c16ea6508be733c91758ecd1,1,1,0,0,0,0,0,0,0,...,23,6760,32.30,0.0,12.64,46.90,0,0.0,0.00,0.0


### Link Labels to Stats

In [15]:
import numpy as np
def make_x(commit_stats):
    return [
        np.log1p(commit_stats['binFiles']),
        np.log1p(commit_stats['textFiles']),
        np.log1p(commit_stats['textLines']),
        np.log1p(commit_stats['javascriptLines']),
        np.log1p(commit_stats['rustLines']),
        np.log1p(commit_stats['markdownLines']),
        np.log1p(commit_stats['jsonLines']),
        np.log1p(commit_stats['imgFiles']),
        np.log1p(commit_stats['lockLines']),
        np.log1p(commit_stats['yarnLines']),
        np.log1p(commit_stats['htmlLines'])
    ]


In [16]:
stats = {}
for commit in all_stats:
    stats[commit['commit']] = commit

X = []
Y = []
a = 0
for row in data.to_numpy():
    commit_id = row[0]
    row_labels = row[slice(2, 12)]
    x = make_x(stats[commit_id])
    #print(x)
    #print(a)
    X.append(make_x(stats[commit_id]))
    Y.append(row_labels.tolist())
    a += 1
#print(a)

#print(Y[slice(2)])
#print(Y[0][0])
#print(X[slice(2)])

In [17]:
from sklearn.datasets import make_multilabel_classification

sample_X, sample_y = make_multilabel_classification(n_classes=3, random_state=0)
print(sample_y[slice(2)])
print(Y[slice(2)])

[[0 1 0]
 [0 1 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier

print(len(X))
print(len(Y))

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(7, 8), random_state = 1)
clf.fit(X, Y)
score = clf.score(X, Y)
Y_pred = clf.predict(X)
print(score)
# print(Y_pred)

140
140
0.9214285714285714


/home/bob/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = \
    train_test_split(X, Y, test_size=0.33, random_state=3)

clf = MLPClassifier(solver='lbfgs', alpha=1e-6, hidden_layer_sizes=(8, 8),
                    random_state = 1, max_iter=500)
clf.fit(X_train, Y_train)
score = clf.score(X_test, Y_test)
Y_pred = clf.predict(X)
print(score)
print(Y_pred)

0.8297872340425532
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [38]:
#print(X_test)
#print(Y_pred)
pairs = []
for i in range(0,len(X_test)):
    #print(Y_pred[i].tolist())
    pairs.append(Y_pred[i].tolist() + X_test[i])
# print(pairs)

print(len(X_train))
import pandas as pd
pd.DataFrame(pairs).to_csv("/home/bob/test.tsv", sep="\t")

93
